In [1]:
import pandas as pd
import numpy as np
import ast
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk
#import 

In [2]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
df=pd.read_csv('../../downloads/GoE_Dataset (1).csv')#../../downloads/Clean_data_GK.csv

In [3]:
df.head(2)

,ID,Title,Pillar,Instructor,Categories,Series,Difficulty,Difficulty_Num,Duration,Duration_Num,...,Type,Audio,Recipe,Text,Video,Visualization,Active,Beauty,Nutrition,Rest
0,438,A comprehensive guide to essential vitamins an...,Nutrition,NaN,Education,NaN,Intermediate,2,6-15 min,3,...,"Text, Video",0,0,1,1,0,0,0,1,0
1,240,A comprehensive guide to nutritional balance,Nutrition,NaN,Education,NaN,Intermediate,2,6-15 min,3,...,"Text, Video",0,0,1,1,0,0,0,1,0


In [4]:
df.shape

(447, 24)

In [5]:
df.dtypes

ID                        int64
Title                    object
Pillar                   object
Instructor               object
Categories               object
Series                   object
Difficulty               object
Difficulty_Num            int64
Duration                 object
Duration_Num              int64
Require_Movement         object
Require_Movement_Num    float64
Tags                     object
Source                   object
Type                     object
Audio                     int64
Recipe                    int64
Text                      int64
Video                     int64
Visualization             int64
Active                    int64
Beauty                    int64
Nutrition                 int64
Rest                      int64
dtype: object

In [6]:
df.isnull().sum().sort_values(ascending=False)

Require_Movement        259
Require_Movement_Num    259
Instructor              188
Series                  188
Categories               23
Difficulty               20
Duration                  3
ID                        0
Recipe                    0
Nutrition                 0
Beauty                    0
Active                    0
Visualization             0
Video                     0
Text                      0
Tags                      0
Audio                     0
Type                      0
Source                    0
Title                     0
Duration_Num              0
Difficulty_Num            0
Pillar                    0
Rest                      0
dtype: int64

In [7]:
print(df.Categories.unique())
print(df.Type.unique())
print(df.Duration.unique())
print(df.Pillar.unique())

['Education' 'Fitness' 'Affirmation' 'Meditation and mindfulness'
 'Nature sounds and ambient music' 'Recipes' 'Get to know the technique'
 nan 'Get started, Short and sweet, Fitness mojo'
 'Get started, Just your body, Fitness mojo'
 'Get started, Short and sweet' 'Just your body, Fitness mojo'
 'On the floor, Get started' 'Breathing exercises and techniques'
 'Short and sweet, Healthy desserts' 'Dance and aerobics' 'Check why'
 'Goe academy' 'Take a breath, Mind and body'
 'Take a breath, Mind and body, Relax' 'Take a breath, Relax'
 'Take a breath, Relax, Mind and body'
 'Short and sweet, Healthy desserts, Quick and easy' 'Stretching, Yoga'
 'Just your body, On the floor, Sweet and short'
 'Just your body, Push yourself' 'Meditation, Mind and body'
 'Take a breath' 'Mind and body, My evening routine'
 'Push yourself, Just your body, Get energised'
 'Push yourself, Just your body' 'Get energised'
 'Yoga and stretching for relaxation' 'Short and sweet, Healthy breakfast'
 'Fitness moj

In [8]:
df.iloc[35].Title

'Energizing zumba dance workout'

### I intend to use -
1) first use cosine similiarity on title,type,category, and pillar for recommendation
2)  lda 

In [21]:
#tree.draw()
df.fillna('',inplace=True)
#df.isnull().sum()


In [10]:
def transform_text(text):
    
    sw=stopwords.words('english')
    lem= WordNetLemmatizer()
    #tokenize
    text=str(text).lower()
    desc_tok=word_tokenize(text)
    # remove stop words
    filtered_desc=[t for t in desc_tok if t not in sw and not t=='Unknown' ] # unknown for type
    
    #lemmatize
    lem_words=[lem.lemmatize(w) for w in filtered_desc]
    return ' '.join(lem_words)


In [24]:
df['text']=df['Title'] +' '+ df['Pillar'] +' '+df['Type'] +' '+ df['Categories']+' '+df['Difficulty']+ ' '+ df['Duration']+ ' '+ df['Tags']
df['text_tk']=df['text'].apply(transform_text)
df['text_tk'].iloc[9]


'affirmation building confidence beauty audio affirmation beginner 1-5 min growth , personal development , self-esteem , motivation , building , empowerment , confidence , positive'

In [12]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer,TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [25]:
vec=TfidfVectorizer()

#tfidfmatrix=vec.fit_transform(df['title_treated'])
tfidfmatrix=vec.fit_transform(df['text_tk'])

In [14]:
#recommend exercises
def recomm_exercise(text):
    text_vec=vec.transform([text])
    sim_scores=cosine_similarity(text_vec,tfidfmatrix)
    # Get the top 10 most similar exercises
    top_exercises_indices = sim_scores[0].argsort()[-5:][::-1]
    top_exercises = df.iloc[top_exercises_indices]
    
    return top_exercises.Title
    

In [27]:
recomms=recomm_exercise('nutrition')
recomms

160                    Understanding macronutrients
161                    Understanding micronutrients
1      A comprehensive guide to nutritional balance
75          Mastering the art of balanced nutrition
2       A comprehensive guide to nutritional basics
Name: Title, dtype: object

## To be continued